# Importing Libraries and Defining Parameters
- In this section, we import the necessary libraries and define the initial parameters, such as image size, batch size, and the data directory.

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 256, 256
batch_size = 32
data_dir = "../cherry-leaves"


### Configuring ImageDataGenerator
- We configure the ImageDataGenerator to perform data augmentation and split the data into training and validation sets.

In [9]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 80% training, 20% validation
)

### Creating Data Generators for Training and Validation
- Here, we create the generators that will read images directly from the directory, applying preprocessing and splitting them into training and validation sets.

In [10]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Binary classification: healthy vs. powdery mildew
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False
)


Found 174 images belonging to 2 classes.
Found 42 images belonging to 2 classes.


### Defining the CNN Architecture
- We define the architecture of the CNN model, consisting of convolutional layers, pooling layers, flattening, and dense layers, with dropout to prevent overfitting.

In [11]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


### Model Compilation and Summary
- We compile the model using the Adam optimizer and the binary_crossentropy loss function. Then, we display a summary of the architecture.

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,105 (56.61 MB)

 Trainable params: 14,839,105 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

### Model Training
- In this step, we train the model using the data generators for training and validation. The number of epochs can be adjusted as needed.

In [13]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.4759 - loss: 1.9029 - val_accuracy: 0.5238 - val_loss: 0.7148
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5074 - loss: 0.7217 - val_accuracy: 0.5238 - val_loss: 0.6779
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5560 - loss: 0.6564 - val_accuracy: 0.5238 - val_loss: 0.6634
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6360 - loss: 0.6563 - val_accuracy: 0.5238 - val_loss: 0.6451
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6730 - loss: 0.5708 - val_accuracy: 0.6429 - val_loss: 0.6236
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6818 - loss: 0.5961 - val_accuracy: 0.8095 - val_loss: 0.5590
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.6870 - loss: 0.5365 - val_accuracy: 0.8571 - val_loss: 0.4916
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.7243 - loss: 0.5641 - val_accuracy: 0.5476 - val_loss: 0.6340
Epoch 9/

### Saving the Trained Model
- After training, we save the model for later use in the dashboard or other applications.

In [14]:
import os

# Define the absolute path to save the model in the correct directory
base_path = os.path.abspath(os.path.join(os.getcwd(), "..", "src"))
os.makedirs(base_path, exist_ok=True)  # Ensure the directory exists

model_path = os.path.join(base_path, "model.h5")
model.save(model_path)  # Save the trained model

print(f"✅ Model successfully saved at {model_path}")

✅ Model successfully saved at /workspaces/Mildew-Detector/src/model.h5
